<a href="https://colab.research.google.com/github/sheetal1996/Eco-lution/blob/main/575DataCleanign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import random
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [46]:
#https://www.geeksforgeeks.org/ways-to-import-csv-files-in-google-colab/
from google.colab import files
 
 
uploaded = files.upload()

Saving owid-co2-data.csv to owid-co2-data (4).csv


In [47]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['owid-co2-data.csv']))
print(df. columns) 

Index(['iso_code', 'country', 'year', 'co2', 'co2_per_capita', 'trade_co2',
       'cement_co2', 'cement_co2_per_capita', 'coal_co2',
       'coal_co2_per_capita', 'flaring_co2', 'flaring_co2_per_capita',
       'gas_co2', 'gas_co2_per_capita', 'oil_co2', 'oil_co2_per_capita',
       'other_industry_co2', 'other_co2_per_capita', 'co2_growth_prct',
       'co2_growth_abs', 'co2_per_gdp', 'co2_per_unit_energy',
       'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_co2', 'cumulative_cement_co2',
       'cumulative_coal_co2', 'cumulative_flaring_co2', 'cumulative_gas_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'trade_co2_share',
       'share_global_co2', 'share_global_cement_co2', 'share_global_coal_co2',
       'share_global_flaring_co2', 'share_global_gas_co2',
       'share_global_oil_co2', 'share_global_other_co2',
       'share_global_cumulative_co2', 'share_global_cumulative_cement_co2',
       'share_global_cumulative_c

In [48]:
df.shape

(25989, 60)

In [49]:
new_df = df.loc[(df['year'] >= 1950)]

In [50]:
#Selecting some columns:
selected_columns = ["country","year","co2", "cement_co2","coal_co2", "gas_co2",
                    "oil_co2","primary_energy_consumption","population"]

new_df_x = new_df[selected_columns] # Filtering out the selected columns

new_df_x.shape

(16476, 9)

In [34]:
new_df_x_null = new_df_x[new_df_x["primary_energy_consumption"].isnull()][["year","primary_energy_consumption"]]

In [ ]:
newpop_df_x_null = new_df_x[new_df_x["population"].isnull()][["year","population"]]

In [ ]:
newpop_df_x_null

In [ ]:
for i in newpop_df_x_null.index:
  newpop_df_x.at[i, "co2"] = n_df_x.at[555, "co2"] + n_df_x.at[23059, "co2"]
n_df_x.loc[558:23059][["country", "year", "co2"]]

> Step 1: We have to find a way to fill the null values.
If we are doing this, we are losing 2020 data as well. Even if we are directly removing the null values using dropna(), we are losing even higher data.


#Urvi


In [ ]:
#n_df = df.loc[(df['year'] >= 1950)]

In [ ]:
#n_df.shape

(16476, 60)

In [ ]:
#n_df.isna().sum()

In [ ]:
#n_df.columns

In [ ]:
#Selecting some columns:
# selected_columns = ["country","year","co2", "cement_co2","coal_co2","gas_co2",
#                     "oil_co2","other_industry_co2","primary_energy_consumption","population"]

# n_df_x = n_df[selected_columns] # Filtering out the selected columns

# n_df_x

In [51]:
new_df_x.isna().sum()

country                          0
year                             0
co2                            576
cement_co2                    5981
coal_co2                      6718
gas_co2                       8836
oil_co2                        644
primary_energy_consumption    7131
population                    1355
dtype: int64

In [52]:
new_df_x_co2 = new_df_x[new_df_x["co2"].isnull()][["year","co2"]]

In [53]:
new_df_x_co2

,year,co2
556,2008,NaN
557,2009,NaN
558,2010,NaN
559,2011,NaN
560,2012,NaN
...,...,...
23054,1997,NaN
23055,1998,NaN
23056,1999,NaN
23057,2000,NaN


In [54]:
for i in new_df_x_co2.index:
  new_df_x.at[i, "co2"] = new_df_x.at[555, "co2"] + new_df_x.at[23059, "co2"]
new_df_x.loc[555:23059][["country", "year", "co2"]]

,country,year,co2
555,Antarctica,2007,0.011
556,Antarctica,2008,0.172
557,Antarctica,2009,0.172
558,Antarctica,2010,0.172
559,Antarctica,2011,0.172
...,...,...,...
23055,Timor,1998,0.172
23056,Timor,1999,0.172
23057,Timor,2000,0.172
23058,Timor,2001,0.172


In [55]:
new_df_x.isnull().sum()

country                          0
year                             0
co2                              0
cement_co2                    5981
coal_co2                      6718
gas_co2                       8836
oil_co2                        644
primary_energy_consumption    7131
population                    1355
dtype: int64

In [63]:
new_df_x_oco2 = new_df_x[new_df_x["oil_co2"].isnull()][["year","oil_co2"]]

In [64]:
new_df_x_oco2

,year,oil_co2
556,2008,NaN
557,2009,NaN
558,2010,NaN
559,2011,NaN
560,2012,NaN
...,...,...
23055,1998,NaN
23056,1999,NaN
23057,2000,NaN
23058,2001,NaN


In [65]:
for i in new_df_x_oco2.index:
  new_df_x.at[i, "oil_co2"] = new_df_x.at[555, "oil_co2"] + new_df_x.at[23275, "oil_co2"]
new_df_x.loc[555:23275][["country", "year", "oil_co2"]]

,country,year,oil_co2
555,Antarctica,2007,0.011
556,Antarctica,2008,1.627
557,Antarctica,2009,1.627
558,Antarctica,2010,1.627
559,Antarctica,2011,1.627
...,...,...,...
23271,Trinidad and Tobago,1960,1.020
23272,Trinidad and Tobago,1961,2.957
23273,Trinidad and Tobago,1962,5.522
23274,Trinidad and Tobago,1963,1.627


In [66]:
new_df_x.isnull().sum()

country                          0
year                             0
co2                              0
cement_co2                    5981
coal_co2                      6718
gas_co2                       8836
oil_co2                          0
primary_energy_consumption    7131
population                    1355
dtype: int64

In [67]:
from sklearn.impute import KNNImputer

In [68]:
knn_imp = KNNImputer(n_neighbors = 5)

In [70]:
new_df_x[['gas_co2']] = knn_imp.fit_transform(new_df_x[['gas_co2']])

In [71]:
new_df_x.isnull().sum()

country                          0
year                             0
co2                              0
cement_co2                    5981
coal_co2                      6718
gas_co2                          0
oil_co2                          0
primary_energy_consumption    7131
population                    1355
dtype: int64

#Vriddhi

#Unused code

In [ ]:
#Let us first find out the columns which have atleast 50% of non-null row
df.columns[df.notnull().mean() >= 0.5]

#Since we need to remove the other columns with less than 50% of non-null rows, we use the 'drop' function
#we assign the resulting dataframe to a new variable name 'df_dropcol'
df_dropcol = df.drop(df.columns[df.notnull().mean() < 0.5],axis=1)

#Now, if we see again the information about this new df_dropcol dataframe (by using the info function), 
#we can see that we still have null values. So, in order to remove these null values, we can use dropna() function on df_dropcol.

df_dropcol.dropna(inplace=True)
df_dropcol.shape


df_gdp_2018 = df_dropcol.loc[df_dropcol['year']==2020] # finds all the countries with data based on 201

#df_gdp_top10 = df_gdp_2018.sort_values(by=['gdp'], ascending = False).head(10) #arranges the countries based on their GDP in 2020

#print(df_gdp_2018['country']) # prints the top 10 countries with highest GDP8
print(df_gdp_2018.to_markdown())


df_nona = df.dropna()

#Investigating the shape of 'df_nona'
df_nona.shape

#df_nona['year'].max() # This line of code will return the most recent year in the data set
df_dropcol['year'].max()


#Selecting some columns:
selected_columns = ["country","year","co2", "cement_co2","coal_co2","gas_co2",
                    "oil_co2","other_industry_co2","primary_energy_consumption","population"]

co2_df = df[selected_columns] # Filtering out the selected columns
co2_df

#aggregating the data by years:
co2_df.groupby(['country', 'year']).sum()

#Top 30 countries of the world by emission
co2_top = co2_df.groupby(by = "country").sum()["co2"]
co2_top = co2_top.sort_values(ascending=False) 
co2_top.head(30)

#A way to fill the data using average:
co2_asia = co2_df[co2_df["country"] == "Asia"] # Filerting out Asian data
co2_europe = co2_df[co2_df["country"] == "Europe"]

co2_asia_null = co2_asia[co2_asia["co2"].isnull()][["year","co2"]] # Checking for NULL values
co2_asia_null


for i in co2_asia_null.index:
  co2_asia.at[i, "co2"] = co2_asia.at[1056, "co2"] + co2_asia.at[1081, "co2"] # Index of the years 1833 and 1857 are 1056 and 1081 respectively
co2_asia.loc[1056:1081][["country", "year", "co2"]]